Continuamos con el modelo __Random Forest__:

El modelo _Random Forest_ es un método general de aprendizaje automático que puede realizar tareas de regresión y clasificación. También realiza métodos de reducción de dimensionalidad para resolver valores perdidos, valores atípicos y otros pasos esenciales de exploración de datos.

# Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt 
import seaborn as sns 
from scipy import stats as sts 
from sklearn.model_selection import train_test_split 
from sklearn import model_selection
from sklearn import linear_model
import random 
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.externals import joblib

Cargamos el dataset:

In [ ]:
LOAN_var_norm = pd.read_csv('../data/intermediate/LOAN_var_norm.csv', delimiter =",")
LOAN_var_norm

# Train y Test

Para reducir el coste computacional, hemos ido probando a tirar los modelos con no todos los datos para intentar encontrar un punto a partir del cual los modelos no nos mejoren aún aumentando la cantidad de observaciones. 

Creemos que a más datos mejor, pero que los algoritmos quizás no necesitan tantos datos para obtener buenos resultados, y tirándolos con los más de 400k datos, se nos eternizaba tanto que nunca terminaba de salir, por ejemplo los grids. 

In [1]:
data_train1 = LOAN_var_norm.sample(n=100000, random_state=1234)

NameError: name 'LOAN_var_norm' is not defined

In [ ]:
data_train1.info()

In [ ]:
Y = data_train1['Charged_off']
X = data_train1.drop(['Charged_off'], axis = 1)

In [ ]:
# Vemos que no está 'Charged _off'
X.info()

In [ ]:
Y.tail()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1234)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.head()

Hacemos _undersampling_ porque tenemos datos de sobra como para hacer oversampling. Lo que queremos es bajar la clase que tiene más cantidad, a la que tiene menos. 

Haremos undersampling de los datos de nuestra muestra para balancearlos:

In [ ]:
undersample = RandomUnderSampler(sampling_strategy = 'majority')

In [ ]:
x_under, y_under = undersample.fit_sample(x_train, y_train)

In [ ]:
x_under.shape

In [ ]:
y_under.shape

# Random Forest

In [ ]:
# njobs = 3 para usar 3 procesadores.
RF = RandomForestClassifier(n_estimators = 100 , random_state = 1234, n_jobs = 3)
RF.fit(x_under, y_under)
RF.score(x_under, y_under)
RF_predicted = RF.predict(x_test)
print(classification_report(y_test,RF_predicted))

In [ ]:
print(confusion_matrix(y_test, RF_predicted))

In [ ]:
coef_RF = pd.DataFrame(RF.feature_importances_)
print(coef_RF)

In [ ]:
metrics.plot_roc_curve(RF, x_test, y_test)
plt.show()

Con 300.000 datos (teniendo en cuenta que hacemos under sampling), antes del grid obtenemos un accuracy del 65%, igual que con muestras de menos cantidad de datos. 

In [ ]:
nombre = 'RandomF.sav'
pickle.dump(RF, open(nombre, 'wb'))

### Búsqueda Grid

In [ ]:
param_grid = {'n_estimators': [500, 1000, 1000]}  
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3, n_jobs = 3)
grid.fit(x_under, y_under) 

In [ ]:
print(grid.best_estimator_) 

In [ ]:
# guardamos el mejor modelo
RF_Best = grid.best_estimator_

In [ ]:
grid_predictions = RF_Best.predict(x_test)
print(classification_report(y_test, grid_predictions))

In [ ]:
print(confusion_matrix(y_test, RF_predicted))

Vemos que el modelo no ha mejorado después del grid.